[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fisamz/Repositorio_MCDAA/blob/main/Tarea2/Tarea2.ipynb)

# Tarea 1 — Creación y operaciones básicas con PySpark  
**Alumno:** Fisam Zavala  
**Dataset:** Resultados de futbol y momios de casas de apuestas.
**Fuente:** [European Soccer Database](https://www.kaggle.com/datasets/hugomathien/soccer)


In [ ]:
#%pip install pyspark

In [1]:
import pyspark
pyspark.__version__


'4.1.1'

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Tarea1_DatosMasivos_PySpark")
    .master("local[*]")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

print("Spark version:", spark.version)
spark

Spark version: 4.1.1


In [7]:
#pip install kaggle
#kaggle datasets download -d hugomathien/soccer

In [ ]:
import sqlite3
import pandas as pd

# Conectar a la base
conn = sqlite3.connect("../data/database.sqlite")

# Ver tablas disponibles
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

conn.close()

tables

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [23]:

df_match = pd.read_sql("SELECT * FROM Match", conn)
print(df_match.shape)
df_match.to_csv("../data/match.csv", index=False)


(25979, 115)


In [ ]:

conn = sqlite3.connect("../data/database.sqlite")

# ---------
# Match
# ---------
df_match = pd.read_sql("SELECT * FROM Match", conn)
print("Match:", df_match.shape)
df_match.to_csv("../data/match.csv", index=False)

# ---------
# Team
# ---------
df_team = pd.read_sql("SELECT * FROM Team", conn)
print("Team:", df_team.shape)
df_team.to_csv("../data/team.csv", index=False)

# ---------
# League
# ---------
df_league = pd.read_sql("SELECT * FROM League", conn)
print("League:", df_league.shape)
df_league.to_csv("../data/league.csv", index=False)

# ---------
# Country
# ---------
df_country = pd.read_sql("SELECT * FROM Country", conn)
print("Country:", df_country.shape)
df_country.to_csv("../data/country.csv", index=False)

# ---------
# Player
# ---------
df_player = pd.read_sql("SELECT * FROM Player", conn)
print("Player:", df_player.shape)
df_player.to_csv("../data/player.csv", index=False)

# ---------
# Player Attributes
# ---------
df_player_attr = pd.read_sql("SELECT * FROM Player_Attributes", conn)
print("Player_Attributes:", df_player_attr.shape)
df_player_attr.to_csv("../data/player_attributes.csv", index=False)

# ---------
# Team Attributes
# ---------
df_team_attr = pd.read_sql("SELECT * FROM Team_Attributes", conn)
print("Team_Attributes:", df_team_attr.shape)
df_team_attr.to_csv("../data/team_attributes.csv", index=False)

conn.close()

print("Todas las tablas fueron exportadas")


Match: (25979, 115)
Team: (299, 5)
League: (11, 3)
Country: (11, 2)
Player: (11060, 7)
Player_Attributes: (183978, 42)
Team_Attributes: (1458, 25)
Todas las tablas fueron exportadas


In [34]:
from pyspark.sql import functions as F

match = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("../data/match.csv")
)

team = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("../data/team.csv")
)

print("Partidos:", match.count())
match.printSchema()


Partidos: 27383
root
 |-- id: string (nullable = true)
 |-- country_id: string (nullable = true)
 |-- league_id: integer (nullable = true)
 |-- season: string (nullable = true)
 |-- stage: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- match_api_id: integer (nullable = true)
 |-- home_team_api_id: integer (nullable = true)
 |-- away_team_api_id: integer (nullable = true)
 |-- home_team_goal: integer (nullable = true)
 |-- away_team_goal: integer (nullable = true)
 |-- home_player_X1: double (nullable = true)
 |-- home_player_X2: double (nullable = true)
 |-- home_player_X3: double (nullable = true)
 |-- home_player_X4: double (nullable = true)
 |-- home_player_X5: double (nullable = true)
 |-- home_player_X6: double (nullable = true)
 |-- home_player_X7: double (nullable = true)
 |-- home_player_X8: double (nullable = true)
 |-- home_player_X9: double (nullable = true)
 |-- home_player_X10: double (nullable = true)
 |-- home_player_X11: double (nullable = true)
 

In [36]:
match2 = (
    match
    .withColumn("total_goals", F.col("home_team_goal") + F.col("away_team_goal"))
    .withColumn("goal_diff", F.col("home_team_goal") - F.col("away_team_goal"))
    .withColumn("is_home_win", (F.col("home_team_goal") > F.col("away_team_goal")).cast("int"))
)

match2.select(
    "date", "home_team_goal", "away_team_goal",
    "total_goals", "goal_diff", "is_home_win"
).show(10, truncate=False)



+-------------------+--------------+--------------+-----------+---------+-----------+
|date               |home_team_goal|away_team_goal|total_goals|goal_diff|is_home_win|
+-------------------+--------------+--------------+-----------+---------+-----------+
|2008-08-17 00:00:00|1             |1             |2          |0        |0          |
|2008-08-16 00:00:00|0             |0             |0          |0        |0          |
|2008-08-16 00:00:00|0             |3             |3          |-3       |0          |
|2008-08-17 00:00:00|5             |0             |5          |5        |1          |
|2008-08-16 00:00:00|1             |3             |4          |-2       |0          |
|2008-09-24 00:00:00|1             |1             |2          |0        |0          |
|2008-08-16 00:00:00|2             |2             |4          |0        |0          |
|2008-08-16 00:00:00|1             |2             |3          |-1       |0          |
|2008-08-16 00:00:00|1             |0             |1  

In [37]:
match2.select(
    "home_team_goal",
    "away_team_goal",
    "total_goals",
    "goal_diff"
).describe().show()


+-------+------------------+------------------+------------------+-------------------+
|summary|    home_team_goal|    away_team_goal|       total_goals|          goal_diff|
+-------+------------------+------------------+------------------+-------------------+
|  count|             25979|             25979|             25979|              25979|
|   mean|1.5445937103044767|1.1609376804341969|2.7055313907386735|0.38365602987027986|
| stddev|1.2971582225804408|1.1421103393870686| 1.672456180315952| 1.7824032339537441|
|    min|                 0|                 0|                 0|                 -9|
|    max|                10|                 9|                12|                 10|
+-------+------------------+------------------+------------------+-------------------+



In [38]:
match2.filter(F.col("total_goals") >= 5) \
      .select("date", "home_team_goal", "away_team_goal", "total_goals") \
      .show(10, truncate=False)


+-------------------+--------------+--------------+-----------+
|date               |home_team_goal|away_team_goal|total_goals|
+-------------------+--------------+--------------+-----------+
|2008-08-17 00:00:00|5             |0             |5          |
|2008-11-01 00:00:00|4             |1             |5          |
|2008-11-08 00:00:00|2             |3             |5          |
|2008-11-14 00:00:00|3             |2             |5          |
|2008-11-22 00:00:00|3             |2             |5          |
|2008-11-23 00:00:00|2             |3             |5          |
|2008-11-29 00:00:00|3             |2             |5          |
|2008-11-29 00:00:00|4             |2             |6          |
|2008-11-29 00:00:00|3             |3             |6          |
|2008-12-06 00:00:00|5             |1             |6          |
+-------------------+--------------+--------------+-----------+
only showing top 10 rows


In [39]:
match2.groupBy("is_home_win").count().show()

total = match2.count()

match2.groupBy("is_home_win") \
      .count() \
      .withColumn("percentage", F.round(F.col("count") / total * 100, 2)) \
      .show()


+-----------+-----+
|is_home_win|count|
+-----------+-----+
|          1|11917|
|          0|14062|
|       NULL| 1404|
+-----------+-----+

+-----------+-----+----------+
|is_home_win|count|percentage|
+-----------+-----+----------+
|          1|11917|     43.52|
|          0|14062|     51.35|
|       NULL| 1404|      5.13|
+-----------+-----+----------+



In [40]:
match2.withColumn("season_year", F.substring("season", 1, 4).cast("int")) \
      .groupBy("season_year") \
      .agg(
          F.avg("total_goals").alias("avg_goals"),
          F.count("*").alias("num_matches")
      ) \
      .orderBy("season_year") \
      .show(15)


+-----------+------------------+-----------+
|season_year|         avg_goals|num_matches|
+-----------+------------------+-----------+
|       NULL|              NULL|       1404|
|       2008| 2.607336139506915|       3326|
|       2009|2.6724458204334365|       3230|
|       2010|2.6837423312883435|       3260|
|       2011|2.7164596273291925|       3220|
|       2012|2.7726993865030676|       3260|
|       2013| 2.766820580474934|       3032|
|       2014|2.6757894736842105|       3325|
|       2015|2.7546602525556225|       3326|
+-----------+------------------+-----------+

